<a href="https://colab.research.google.com/github/sadaha/Coursera_Capstone.-/blob/main/IBM_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IBM Capstone Project**

## Here I will write my report, and code for the IBM capstone project.

Problem Description:

> The city used for the project will be Hamburg. The problem will be looking at types of restaurant in the city, such as Italian, German, Korean, Chinese etc, and trying to cluster the different neighbourhoods by which type of restaunts are most populous in which areas, such that the client, a new restauranteer who wants to open a Korean restaurant, can see areas of potentially high and low competition.



Data Description:

> The data will be the locations and types of restaurants in each city borough, as well as their average rating. To be used to determine how much competition they could provide. The data will be used to look at areas of high and low restaurant density (of all types of cuisine), as well as look at the frequency of korean restaurants present in each city sector. The data will be sourced from Foursquare if I can ever get the site to let me make an account. Else I will find a CSV dataset somewhere.

# Obtaining the Data

Hamburg Areas:

> Here I will scrape the different locations in Hamburg from Wikipedia by quarter, and the postcodes for each area. The area information we will take from the following site https://www.citypopulation.de/en/germany/hamburg/admin/

From this we will take the name of the quarter and the population as most recently measured.

We will then use the ArcGIS to get the latitude and longitude data for each quarter, with which we can query Foursquare to get information.

# Import the libraries

In [ ]:
import pandas as pd
import requests
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

# import k-means for the clustering stage
from sklearn.cluster import KMeans

# Read the webpage and check the response

In [ ]:
url = "https://www.citypopulation.de/en/germany/hamburg/admin/"
hamburg_url = requests.get(url)
hamburg_url

<Response [200]>

Check how the data is read in

In [ ]:
hamburg_df_raw = pd.read_html(hamburg_url.text)
hamburg_df_raw

[                  Name   Status  ... PopulationEstimate2019-12-31 Unnamed: 7
 0               Altona  Borough  ...                       275265          →
 1      Altona-Altstadt  Quarter  ...                        29170          →
 2          Altona-Nord  Quarter  ...                        24536          →
 3           Bahrenfeld  Quarter  ...                        30203          →
 4           Blankenese  Quarter  ...                        13730          →
 ..                 ...      ...  ...                          ...        ...
 108          Volksdorf  Quarter  ...                        20978          →
 109           Wandsbek  Quarter  ...                        36444          →
 110     Wellingsbüttel  Quarter  ...                        10848          →
 111  Wohldorf-Ohlstedt  Quarter  ...                         4650          →
 112            Hamburg     City  ...                      1899160        NaN
 
 [113 rows x 8 columns],   Gender (E 2019)  Gender (E 2019).1


The data is read in two tables, the first one is the one that is relevant to us now. So remove the second list object.

In [ ]:
hamburg_df_raw = hamburg_df_raw[0]
hamburg_df_raw

,Name,Status,PopulationEstimate2002-12-31,PopulationEstimate2007-12-31,PopulationEstimate2012-12-31,PopulationEstimate2017-12-31,PopulationEstimate2019-12-31,Unnamed: 7
0,Altona,Borough,...,257299,257412,270263,275265,→
1,Altona-Altstadt,Quarter,27550,26918,28174,29034,29170,→
2,Altona-Nord,Quarter,21662,22521,21759,22137,24536,→
3,Bahrenfeld,Quarter,26343,27193,27165,31047,30203,→
4,Blankenese,Quarter,13452,13435,13016,13407,13730,→
...,...,...,...,...,...,...,...,...
108,Volksdorf,Quarter,18818,19362,20115,20625,20978,→
109,Wandsbek,Quarter,32583,33034,33131,34469,36444,→
110,Wellingsbüttel,Quarter,9432,9829,10263,10506,10848,→
111,Wohldorf-Ohlstedt,Quarter,4189,4405,4423,4656,4650,→


We only need the "Name", "State" and last Population collumn, so remove the rest.

In [ ]:
hamburg_df = hamburg_df_raw.drop(['PopulationEstimate2002-12-31', 'PopulationEstimate2007-12-31','PopulationEstimate2012-12-31', 'PopulationEstimate2017-12-31', 'Unnamed: 7'], axis=1)
hamburg_df

,Name,Status,PopulationEstimate2019-12-31
0,Altona,Borough,275265
1,Altona-Altstadt,Quarter,29170
2,Altona-Nord,Quarter,24536
3,Bahrenfeld,Quarter,30203
4,Blankenese,Quarter,13730
...,...,...,...
108,Volksdorf,Quarter,20978
109,Wandsbek,Quarter,36444
110,Wellingsbüttel,Quarter,10848
111,Wohldorf-Ohlstedt,Quarter,4650


Curently the data frame has Quarters and Boroughs, we want to remove the Borough values

In [ ]:
indexNames = hamburg_df[ hamburg_df['Status'] == 'Borough' ].index
hamburg_df.drop(indexNames , inplace=True)
hamburg_df

,Name,Status,PopulationEstimate2019-12-31
1,Altona-Altstadt,Quarter,29170
2,Altona-Nord,Quarter,24536
3,Bahrenfeld,Quarter,30203
4,Blankenese,Quarter,13730
5,Groß Flottbek,Quarter,11111
...,...,...,...
108,Volksdorf,Quarter,20978
109,Wandsbek,Quarter,36444
110,Wellingsbüttel,Quarter,10848
111,Wohldorf-Ohlstedt,Quarter,4650


The last value is the city itself, so remove that too

In [ ]:
hamburg_df.drop(hamburg_df.tail(1).index,inplace=True)
hamburg_df

,Name,Status,PopulationEstimate2019-12-31
1,Altona-Altstadt,Quarter,29170
2,Altona-Nord,Quarter,24536
3,Bahrenfeld,Quarter,30203
4,Blankenese,Quarter,13730
5,Groß Flottbek,Quarter,11111
...,...,...,...
107,Tonndorf,Quarter,15148
108,Volksdorf,Quarter,20978
109,Wandsbek,Quarter,36444
110,Wellingsbüttel,Quarter,10848


Now we can find the latitude and longitude data for each Quarter and add them to the dataframe. To start we install arcgid.

In [ ]:
pip install arcgis

In [ ]:
from arcgis.geocoding import geocode
from arcgis.gis import GIS
gis = GIS()

Make a function to return the lat and lon information from the quarter name

In [ ]:
def get_x_y(address1):
   lat_coords = 0
   lng_coords = 0
   g = geocode(address='{}, Hamburg, Germany, GER'.format(address1))[0]
   lng_coords = g['location']['x']
   lat_coords = g['location']['y']
   return str(lat_coords) +","+ str(lng_coords)

Separate the location names into a seperate dataframe

In [ ]:
geo_coordinates = hamburg_df['Name']  
geo_coordinates

Get the Latitude and Longitude data for each area in hamburg

In [ ]:
coordinates_latlong = geo_coordinates.apply(lambda x: get_x_y(x))
coordinates_latlong

1        53.55078000000003,9.940710000000024
2        53.56804000000005,9.948140000000024
3         53.56466000000006,9.90166000000005
4        53.56314000000003,9.812320000000057
5        53.56564000000003,9.877840000000049
                       ...                  
107    53.586970000000065,10.127130000000022
108     53.65131000000008,10.167030000000068
109     53.57205000000005,10.067070000000058
110    53.640640000000076,10.083020000000033
111     53.69557000000003,10.139000000000067
Name: Name, Length: 105, dtype: object

Separate the data into two columns, one for latitude and one for longitude.

In [ ]:
lat_Hamburg = coordinates_latlong.apply(lambda x: x.split(',')[0])
#lat_Hamburg
lon_Hamburg = coordinates_latlong.apply(lambda x: x.split(',')[1])
lon_Hamburg

1       9.940710000000024
2       9.948140000000024
3        9.90166000000005
4       9.812320000000057
5       9.877840000000049
              ...        
107    10.127130000000022
108    10.167030000000068
109    10.067070000000058
110    10.083020000000033
111    10.139000000000067
Name: Name, Length: 105, dtype: object

Make a single dataframe with all the data

In [ ]:
Hamburg_df_complete = pd.concat([hamburg_df,lat_Hamburg.astype(float), lon_Hamburg.astype(float)], axis=1)
Hamburg_df_complete.columns= ['Name','Status','PopulationEstimate2019-12-31','latitude','longitude']
Hamburg_df_complete

,Name,Status,PopulationEstimate2019-12-31,latitude,longitude
1,Altona-Altstadt,Quarter,29170,53.55078,9.94071
2,Altona-Nord,Quarter,24536,53.56804,9.94814
3,Bahrenfeld,Quarter,30203,53.56466,9.90166
4,Blankenese,Quarter,13730,53.56314,9.81232
5,Groß Flottbek,Quarter,11111,53.56564,9.87784
...,...,...,...,...,...
107,Tonndorf,Quarter,15148,53.58697,10.12713
108,Volksdorf,Quarter,20978,53.65131,10.16703
109,Wandsbek,Quarter,36444,53.57205,10.06707
110,Wellingsbüttel,Quarter,10848,53.64064,10.08302


Now to visualise a map of Hamburg. To allow visualisation of the data

In [ ]:
Hamburg = geocode(address='Hamburg, Germany, GER')[0]
Hamburg_lng_coords = Hamburg['location']['x']
Hamburg_lat_coords = Hamburg['location']['y']
#Hamburg_lng_coords
Hamburg_lat_coords

53.553750000000036

In [ ]:
# Create a map of Hamburg as an object, use the first entry in the datafreame as reference
map = folium.Map(location=[Hamburg_lat_coords, Hamburg_lng_coords], zoom_start=12)
map

# adding markers to map
for latitude, longitude, borough, town in zip(Hamburg_df_complete['latitude'], Hamburg_df_complete['longitude'], Hamburg_df_complete['Name'], Hamburg_df_complete['PopulationEstimate2019-12-31']):
    label = '{}, {}'.format(town, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='red',
        fill=True
        ).add_to(map)  
    
map

# Finding restaurants in Hamburg

Now to use Foursquare (if it works) to find the restaurant locations in hamburg

In [ ]:
CLIENT_ID = 'LDIJF4KI5VGMMA3NNDLFZWHR12TCMNTUL0TUC3QPZ3SJD040' 
CLIENT_SECRET = '0DXHVDFCZXNXFSLOFGOONJSS35KH4NAZXZN2AAAX5GCZVVTH'
VERSION = '20180605' # Foursquare API version

In [ ]:
LIMIT=100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups']['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
venues_in_Hamburg = getNearbyVenues(Hamburg_df_complete['Name'], Hamburg_df_complete['latitude'], Hamburg_df_complete['longitude'])

Altona-Altstadt


KeyError: ignored

# Conclusion and results

I am having trouble connecting to Foursquare because of some signup bug, so I'm hoping that the work thus far is enough to get enough marks to pass. If I had managed to not have some bugs in the Foursquare signup, I would've retrieved all the restaurants in hamburg, and put them into each borough cluster.

I would then look at specifically Korean restaurants as that's what the client wants to open.

I would visualise, maybe with a heatmap, the areas in Hamburg that have the least Korean restaurants.

I would also visualise with another heatmap how many restaurants each area has.

This would allow the employer to see both areas of high Korean restaurant density, as well as high general restaurant intensity. This would allow insight into where to open the restuarant.

I would finish by making a heatmap of population density for each area, using the population data for each borough. I would then create two final heatmaps, measuring korean restaurants per 1000 residents, and general restaurants per 1000 people. This would allow The customer to make informed decisions. 

I would use K means clustering to identify clusters of restuarants in especially dense areas, to show the client how the restaurant distribution is spread out over Hamburg.

# Conclusion

I would conclude the work by advising the client to not open a restaruant as we're in the middle of a pandemic and I don't think it'd be a good idea right now.